# 🚀 GitHub-Sync Master Control

**Panneau de contrôle centralisé pour système GitHub-Sync**

- Configuration stratégies de mise à jour
- Orchestration cycles complets
- Monitoring et rollback
- Interface commande unifiée

---

In [ ]:
# 🔍 DIAGNOSTIC SIMPLE COLAB
import sys
import os

print("🔍 DIAGNOSTIC COLAB SIMPLE")
print("=" * 40)

# 1. Info environnement
print(f"🐍 Python: {sys.version[:20]}...")
print(f"📁 Répertoire: {os.getcwd()}")

# 2. Clone repo si nécessaire (SANS subprocess pour éviter blocage)
if not os.path.exists('src'):
    print("📥 Clonage repo...")
    # Utilisation de ! au lieu de subprocess
    get_ipython().system('git clone https://github.com/stephanedenis/PaniniFS-Research.git')
    os.chdir('PaniniFS-Research')
    print(f"📂 Nouveau répertoire: {os.getcwd()}")

# 3. Vérification structure
print(f"\n📂 STRUCTURE:")
if os.path.exists('src'):
    print(f"✅ src/ existe")
    if os.path.exists('src/github_sync'):
        files = os.listdir('src/github_sync')
        print(f"✅ github_sync/: {len(files)} fichiers")
        if '__init__.py' in files:
            print(f"✅ __init__.py présent")
        else:
            print(f"❌ __init__.py MANQUANT!")
    else:
        print(f"❌ github_sync/ manquant")
else:
    print(f"❌ src/ manquant")

# 4. Setup Python path
src_path = os.path.join(os.getcwd(), 'src')
sys.path.insert(0, src_path)
print(f"\n🔧 Chemin ajouté: {src_path}")

# 5. Test imports SIMPLE
print(f"\n🧪 TEST IMPORTS:")
try:
    import github_sync.config_manager
    print("✅ config_manager: OK")
except Exception as e:
    print(f"❌ config_manager: {e}")

try:
    import github_sync.github_loader
    print("✅ github_loader: OK")
except Exception as e:
    print(f"❌ github_loader: {e}")

print("\n🎯 Diagnostic terminé!")

In [ ]:
# 🎛️ PANNEAU CONFIGURATION

# Import des classes nécessaires (après diagnostic cellule 1)
from github_sync.config_manager import GitHubSyncConfig
from github_sync.github_loader import GitHubModuleLoader
from github_sync.hot_reload import HotReloadManager
from github_sync.module_updater import ModuleUpdater

# Initialiser gestionnaire configuration
config = GitHubSyncConfig()

print("🔧 CONFIGURATION GITHUB-SYNC")
print("=" * 50)

# Afficher configuration actuelle
summary = config.get_config_summary()
print(f"📊 Résumé Configuration:")
print(f"├── Stratégie: {summary['strategy'].upper()}")
print(f"├── Auto-rollback: {'✅' if summary['auto_rollback'] else '❌'}")
print(f"├── Intervalle vérif: {summary['check_interval_min']} min")
print(f"├── Modules cibles: {summary['target_modules_count']}")
print(f"├── Mode debug: {'✅' if summary['debug_mode'] else '❌'}")
print(f"└── Dernière MAJ: {summary['last_update']}")

print(f"\n🎯 Configuration prête pour hot-reload!")

In [ ]:
# ? ÉTAT SYSTÈME ET ROLLBACK

# Initialiser gestionnaire de mises à jour (utilise config de cellule précédente)
updater = ModuleUpdater(config)

print("? ÉTAT SYSTÈME GITHUB-SYNC")
print("=" * 50)

# État actuel système
print("\n📊 État Système:")
status = updater.get_update_status()
print(f"├── Stratégie active: {status['strategy']}")
print(f"├── Auto-rollback: {'✅' if status['auto_rollback'] else '❌'}")
print(f"├── Dernière MAJ: {status['last_update'] or 'Jamais'}")
print(f"├── Historique MAJ: {status['update_count']} cycles")
print(f"└── Points rollback: {status['rollback_points']}")

# Points de rollback disponibles
rollback_points = updater.list_available_rollback_points()
if rollback_points:
    print("\n⏰ Points Rollback Disponibles:")
    for i, point in enumerate(rollback_points, 1):
        age = point['age_hours']
        fallback = '✅' if point['fallback_available'] else '❌'
        print(f"{i:2d}. {point['timestamp'][:19]} (il y a {age:.1f}h) - Fallback: {fallback}")
else:
    print("\n⏰ Aucun point de rollback disponible")

print(f"\n🎯 Système surveillé et prêt!")

In [ ]:
# ? DÉMARRAGE HOT-RELOAD

# Initialiser le système hot-reload complet
loader = GitHubModuleLoader(config)
hot_reload = HotReloadManager(config, loader, updater)

print("🚀 DÉMARRAGE HOT-RELOAD GITHUB")
print("=" * 50)

# Configuration du système
setup_result = hot_reload.setup_monitoring()
if setup_result['success']:
    print("✅ Système hot-reload configuré")
    print(f"📁 Modules surveillés: {setup_result['monitored_modules']}")
    print(f"⏱️ Intervalle: {setup_result['check_interval']} minutes")
else:
    print(f"❌ Erreur configuration: {setup_result['error']}")

# Démarrage surveillance
print(f"\n🔄 Démarrage surveillance...")
monitoring_result = hot_reload.start_monitoring()

if monitoring_result['success']:
    print("✅ Hot-reload actif!")
    print(f"📊 Status: {monitoring_result['status']}")
    print(f"🔗 Connexion GitHub: {'✅' if monitoring_result['github_connected'] else '❌'}")
    print(f"📡 Thread monitoring: {'✅' if monitoring_result['thread_active'] else '❌'}")
    
    # Informations détaillées
    details = hot_reload.get_monitoring_details()
    print(f"\n📋 Détails Surveillance:")
    print(f"├── Modules chargés: {details['loaded_modules']}")
    print(f"├── Dernière vérif: {details['last_check']}")
    print(f"├── Mises à jour: {details['updates_applied']}")
    print(f"└── Erreurs: {details['error_count']}")
    
    print(f"\n🎉 SYSTÈME OPÉRATIONNEL!")
    print(f"💡 Les modules seront automatiquement mis à jour depuis GitHub")
    print(f"⚡ Pas besoin de redémarrer Colab lors des mises à jour!")
    
else:
    print(f"❌ Erreur démarrage: {monitoring_result['error']}")
    print(f"🔧 Vérifiez la configuration et la connexion GitHub")

# Commandes de contrôle disponibles
print(f"\n🎛️ COMMANDES DISPONIBLES:")
print(f"├── hot_reload.force_update() - Force une mise à jour")
print(f"├── hot_reload.pause_monitoring() - Met en pause")
print(f"├── hot_reload.resume_monitoring() - Reprend")
print(f"├── hot_reload.get_status() - Affiche l'état")
print(f"└── updater.rollback() - Rollback en cas de problème")

In [ ]:
# 🎛️ PANNEAU DE CONTRÔLE CONTINU

from datetime import datetime
import time

print("🎛️ MONITORING GITHUB-SYNC CONTINU")
print("=" * 50)

# Statut monitoring actuel
status = hot_reload.get_monitoring_status()
print(f"\n📊 STATUT MONITORING:")
print(f"├── Actif: {'✅' if status['monitoring_active'] else '❌'}")
print(f"├── Auto-reload: {'✅' if status['auto_reload_enabled'] else '❌'}")
print(f"├── Queue: {status['queue_length']} modules")
print(f"├── Reload en cours: {'✅' if status['reload_in_progress'] else '❌'}")
print(f"├── Total reloads: {status['total_reloads']}")
print(f"└── Dernier reload: {status['last_reload'] or 'Jamais'}")

# Boutons de contrôle
print(f"\n🎮 COMMANDES MONITORING CONTINU:")
print(f"1. ▶️  DÉMARRER : hot_reload.start_background_monitoring(5)  # 5min intervalle")
print(f"2. ⏹️  ARRÊTER  : hot_reload.stop_background_monitoring()")
print(f"3. 📊 STATUT   : hot_reload.get_monitoring_status()")
print(f"4. 🔄 FORCE    : hot_reload.manual_reload_trigger()")

# Interface de contrôle interactive
print(f"\n⚡ DÉMARRAGE AUTOMATIQUE:")
print(f"Copier-coller une de ces commandes:")
print(f"")
print(f"# Monitoring ultra-rapide (2 min) - pour développement")
print(f"hot_reload.start_background_monitoring(2)")
print(f"")
print(f"# Monitoring standard (5 min) - équilibré")
print(f"hot_reload.start_background_monitoring(5)")
print(f"")
print(f"# Monitoring lent (10 min) - pour production")
print(f"hot_reload.start_background_monitoring(10)")

# Monitoring temps réel (optionnel)
print(f"\n📡 SURVEILLANCE TEMPS RÉEL:")
print(f"Pour voir les mises à jour en temps réel, lancez dans une cellule:")
print(f"")
print(f"```python")
print(f"import time")
print(f"from IPython.display import clear_output")
print(f"")
print(f"while True:")
print(f"    clear_output(wait=True)")
print(f"    status = hot_reload.get_monitoring_status()")
print(f"    print(f'🕐 {{datetime.now().strftime(\"%H:%M:%S\")}} | Monitoring: {{\"✅\" if status[\"monitoring_active\"] else \"❌\"}} | Queue: {{status[\"queue_length\"]}} | Reloads: {{status[\"total_reloads\"]}}')") 
print(f"    time.sleep(10)  # Actualiser toutes les 10 secondes")
print(f"```")

print(f"\n🎯 SYSTÈME PRÊT POUR MONITORING CONTINU!")
print(f"💡 Une fois démarré, les modules se mettront à jour automatiquement depuis GitHub!")
print(f"⚡ Plus besoin de redémarrer Colab lors des mises à jour!")